In [1]:
import numpy as np
import pandas as pd
import datetime
import os
from fastapi import FastAPI, File, UploadFile
import sqlite3
from pathlib import Path
#from tabulate import tabulate
# import psycopg2
import random
import string

In [2]:
#cargamos los datos diarios
product_views=pd.read_csv("product_views.csv")
ads_views=pd.read_csv("ads_views.csv")
product_views.head()
ads_views.head()

,advertiser_id,product_id,type,date
0,KD9PHCBGYFBRI9ET1O9R,kxo57g,impression,25/4/2023
1,LW045DVYSGRD75TK6U54,mxovzl,impression,25/4/2023
2,IOBPI63RBJIHI5FB7U9O,ox5vaz,impression,25/4/2023
3,LW045DVYSGRD75TK6U54,mhvv97,impression,25/4/2023
4,2WPF1NXECF3G6NUMWDO7,ugdfpk,impression,25/4/2023


In [3]:
#nos quedamos únicamente con los advertiser activos
random.seed(4)
active_advertisers = [''.join(random.choices(string.ascii_uppercase + string.digits, k = 20)) for _ in range(20)]
inactive_advertisers = [''.join(random.choices(string.ascii_uppercase + string.digits, k = 20)) for _ in range(5)]
all_advertisers = active_advertisers+inactive_advertisers

In [4]:
#resultados diarios para el modelo 1
product_views_filtrado_aux=product_views['advertiser_id'].isin(active_advertisers) #filtro por los activos 
product_views_filtrado=product_views[product_views_filtrado_aux]
product_views_agrupado=product_views_filtrado.groupby(['date','advertiser_id','product_id']).size().reset_index(name='product_id_count') #agrupamos por fecha, advertiser y product id porque hay casi 70 mil registros repetidos y contamos la cantidad de veces que se repite el registro
product_views_ordenado = product_views_agrupado.sort_values(by=['date', 'advertiser_id','product_id_count'], ascending=[True, True, False]) #ordena por mas vistos
topproduct_diario=product_views_ordenado.groupby(['date', 'advertiser_id']).apply(lambda x: x.nlargest(20, 'product_id_count')).reset_index(drop=True) #Se queda únicamente con los 20 más vistos por advertiser


In [5]:
#resultados diarios para el modelo 2
ads_views_filtrado_aux=ads_views['advertiser_id'].isin(active_advertisers) #filtro por adv actvivos
ads_views_filtrado=ads_views[ads_views_filtrado_aux]
ads_views_agrupado=ads_views_filtrado.groupby(['date','advertiser_id','product_id','type']).size().reset_index(name='product_id_count')#agrupo porque hay registros iguales
ads_views_agrupado.head()
#cálculos auxiliares para el modelo 2
pivot = ads_views_agrupado.pivot_table(index=['date', 'advertiser_id', 'product_id'], columns='type', values='product_id_count', aggfunc='sum')
pivot.columns = ['clicks', 'impressions']
pivot = pivot.fillna(0)
pivot['click_rate'] = pivot['clicks'] / pivot['impressions']
ads_views_agrupado_cr = pivot.reset_index() #nuevo df con columna click rate
#ordenamos los resultados del modelo 2 y nos quedamos con los primeros 20
ads_views_ordenado = ads_views_agrupado_cr.sort_values(by=['date', 'advertiser_id','click_rate'], ascending=[True, True, False])
ctr_results_diario=ads_views_ordenado.groupby(['date', 'advertiser_id']).apply(lambda x: x.nlargest(20, 'click_rate')).reset_index(drop=True)

In [9]:
#creamos una base con resultados consolidados
ctr_results_diario #resultados diario modelo 2
topproduct_diario #resultado diario modelo 1
topproduct_diario['date'] = pd.to_datetime(topproduct_diario['date'], format='%d/%m/%Y')# convertimos columna fecha a fecha
ctr_results_diario['date'] = pd.to_datetime(ctr_results_diario['date'], format='%d/%m/%Y')
ctr_results_diario_aux=ctr_results_diario #agregamos columna de modelo
ctr_results_diario_aux['modelo']='modelo_1'
topproduct_diario_aux=topproduct_diario
topproduct_diario_aux['modelo']='modelo_2'

#ctr_results_diario_aux.drop(['clicks', 'impressions','click_rate'], axis=1, inplace=True) 
#topproduct_diario_aux.drop('product_id_count', axis=1, inplace=True)

resultados_consolidados=pd.concat([ctr_results_diario_aux,topproduct_diario_aux], axis=0, ignore_index=True)
resultados_consolidados



,date,advertiser_id,product_id,modelo
0,2023-05-01,2WPF1NXECF3G6NUMWDO7,lq3nso,modelo_1
1,2023-05-01,2WPF1NXECF3G6NUMWDO7,gkz6wc,modelo_1
2,2023-05-01,2WPF1NXECF3G6NUMWDO7,ipdbkq,modelo_1
3,2023-05-01,2WPF1NXECF3G6NUMWDO7,w3yjck,modelo_1
4,2023-05-01,2WPF1NXECF3G6NUMWDO7,01mgr3,modelo_1
...,...,...,...,...
7995,2023-05-04,Y0W3K7OV6ZLILW96OO3K,7g0p27,modelo_2
7996,2023-05-04,Y0W3K7OV6ZLILW96OO3K,9i1661,modelo_2
7997,2023-05-04,Y0W3K7OV6ZLILW96OO3K,anrfpf,modelo_2
7998,2023-05-04,Y0W3K7OV6ZLILW96OO3K,beso1b,modelo_2


In [7]:
resultados_consolidados.to_csv('resultados_consolidados.csv')

In [10]:
resultados_consolidados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           8000 non-null   datetime64[ns]
 1   advertiser_id  8000 non-null   object        
 2   product_id     8000 non-null   object        
 3   modelo         8000 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 250.1+ KB


**SQL: recomendaciones por modelo**

In [15]:
#Estadísticas de coincidencia entre ambos modelos para los diferentes advs
#agregar una columna al df para ver las coincidencias:
resultados_consolidados_coincidencias=resultados_consolidados.copy()
resultados_consolidados_coincidencias['advertiser_product'] = resultados_consolidados['advertiser_id'] + resultados_consolidados['product_id'] #concateno en una columna product_id y advertiser_id
resultados_consolidados_coincidencias #la idea es que con una consulta SQL se identifiquen las repeticiones por modelo

#esto es para probar, uso solo una fecha
resultados_consolidados_coincidencias_fecha=resultados_consolidados_coincidencias.loc[resultados_consolidados_coincidencias['date'] =="2023-04-28 00:00:00"]


In [14]:
resultados_consolidados_coincidencias.to_csv("resultados_consolidados_coincidencias.csv")

PermissionError: [Errno 13] Permission denied: 'resultados_consolidados_coincidencias.csv'

In [16]:
resultados_consolidados_coincidencias.head()


,date,advertiser_id,product_id,modelo,advertiser_product
0,2023-05-01,2WPF1NXECF3G6NUMWDO7,lq3nso,modelo_1,2WPF1NXECF3G6NUMWDO7lq3nso
1,2023-05-01,2WPF1NXECF3G6NUMWDO7,gkz6wc,modelo_1,2WPF1NXECF3G6NUMWDO7gkz6wc
2,2023-05-01,2WPF1NXECF3G6NUMWDO7,ipdbkq,modelo_1,2WPF1NXECF3G6NUMWDO7ipdbkq
3,2023-05-01,2WPF1NXECF3G6NUMWDO7,w3yjck,modelo_1,2WPF1NXECF3G6NUMWDO7w3yjck
4,2023-05-01,2WPF1NXECF3G6NUMWDO7,01mgr3,modelo_1,2WPF1NXECF3G6NUMWDO701mgr3


In [17]:
resultados_consolidados

,date,advertiser_id,product_id,modelo
0,2023-05-01,2WPF1NXECF3G6NUMWDO7,lq3nso,modelo_1
1,2023-05-01,2WPF1NXECF3G6NUMWDO7,gkz6wc,modelo_1
2,2023-05-01,2WPF1NXECF3G6NUMWDO7,ipdbkq,modelo_1
3,2023-05-01,2WPF1NXECF3G6NUMWDO7,w3yjck,modelo_1
4,2023-05-01,2WPF1NXECF3G6NUMWDO7,01mgr3,modelo_1
...,...,...,...,...
7995,2023-05-04,Y0W3K7OV6ZLILW96OO3K,7g0p27,modelo_2
7996,2023-05-04,Y0W3K7OV6ZLILW96OO3K,9i1661,modelo_2
7997,2023-05-04,Y0W3K7OV6ZLILW96OO3K,anrfpf,modelo_2
7998,2023-05-04,Y0W3K7OV6ZLILW96OO3K,beso1b,modelo_2


In [27]:
import sqlite3

# Conectarse a la base de datos
    conn = sqlite3.connect(':memory:')
    resultados_consolidados.to_sql('tabla_products5', conn, index=False)
    cur = conn.cursor()
    cur.execute(f"SELECT DISTINCT advertiser_id FROM tabla_products5 WHERE modelo = '{modelo}'")
    advertisers = cur.fetchall()
    result = {}
    for adv in advertisers:
        adv_id = adv[0]
        cur.execute(f"SELECT DISTINCT date FROM tabla_products5 WHERE modelo = '{modelo}' AND advertiser_id = '{adv_id}' ORDER BY date")
        dates = cur.fetchall()
        diffs = {}
        for i in range(len(dates)-1):
            date1 = dates[i][0]
            date2 = dates[i+1][0]
            cur.execute(f"SELECT COUNT(DISTINCT product_id) FROM tabla_products5 WHERE modelo = '{modelo}' AND advertiser_id = '{adv_id}'")
            count1 = cur.fetchone()[0]
            cur.execute(f"SELECT COUNT(DISTINCT product_id) FROM tabla_products5 WHERE modelo = '{modelo}' AND advertiser_id = '{adv_id}'")
            count2 = cur.fetchone()[0]
            diffs[date1] = abs(count1 - count2)
        total_diffs = sum(diffs.values())
        result[adv_id] = {"modelo": modelo, "advertiser_id": adv_id, "variaciones": diffs, "total_variaciones": total_diffs}
    result_sorted = sorted(result.values(), key=lambda x: x["total_variaciones"], reverse=True)
    return result_sorted

IndentationError: unexpected indent (Temp/ipykernel_15268/3297477192.py, line 5)

TypeError: variaciones() missing 1 required positional argument: 'modelo'